In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
#Loading the dataset
dataset1 = pd.read_csv("SolarPrediction.csv")

In [ ]:
#Processing the data
import datetime
df= dataset1.copy()
df1= df.copy()
df1['Data'] = pd.to_datetime(df['Data'], format='%m/%d/%Y %H:%M:%S AM')
df1['Data']= df1['Data'].dt.strftime('%m/%d/%Y')
df1 = df1.rename(columns={'Data': 'Date'})
Daily = df1.groupby(by="Date").mean()
Daily.index =pd.to_datetime( Daily.index, format='%m/%d/%Y')
Daily1= Daily.Radiation
Daily1.index =pd.to_datetime( Daily1.index, format='%m/%d/%Y')
Daily2=Daily1.asfreq('d')
df_final= Daily2.interpolate();

In [ ]:
#Visuallising The trends 
import statsmodels.api as sm 
result=sm.tsa.seasonal_decompose(df_final, model='multiplicable')
plt.rcParams["figure.figsize"] = (10,10)
result.plot()

In [ ]:
#Train-Test Split
train_size = int(len(df_final) * 0.67)
test_size = len(df_final) - train_size
train_arima, test_arima =  df_final.iloc[0:train_size], df_final.iloc[train_size:len(df_final)]

# ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm

In [ ]:
#auto_arima to decide the best order
#model = pm.auto_arima(np.transpose(train_arima))
#print(model.summary())

In [ ]:
model = ARIMA(np.transpose(train_arima), order=(1,0,1))
model_fit = model.fit()
ARIMA_training_predictions=model_fit.predict()

In [ ]:
plt.rcParams["figure.figsize"] = (9,6)
plt.plot(ARIMA_training_predictions, label="Model")
plt.plot(train_arima, label="Actual")
plt.title('Training Predictions ARIMA')
plt.xlabel('Date')
plt.xticks(rotation = 45)
plt.ylabel('Radiation')
plt.legend(loc='best')
plt.show()

In [ ]:
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

from sklearn.metrics import mean_squared_error

#rms = mean_squared_error(y_actual, y_predicted, squared=False)

In [ ]:
print("The training MAPE for the ARMA model is:",MAPE(train_arima,ARIMA_training_predictions))
print("The training RMSE for the ARMA model is:",mean_squared_error(train_arima,ARIMA_training_predictions, squared=False))

In [ ]:
history = [x for x in np.transpose(train_arima)]
predictions = list()
for t in range(len(np.transpose(test_arima))):
    model = ARIMA(history, order=(1,0,1))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test_arima[t]
    history.append(obs)#obs originally
    #print('predicted=%f, expected=%f' % (yhat, obs))
ARIMA_testing_predictions= pd.DataFrame(predictions,columns = ['Radiation'], index= test_arima.index)

In [ ]:
plt.plot(ARIMA_testing_predictions, label="Model")
plt.plot(test_arima, label="Actual")
plt.title('Testing Predictions ARIMA')
plt.xlabel('Date')
plt.xticks(rotation = 45)
plt.ylabel('Radiation')
plt.legend(loc='best')
plt.show()

In [ ]:
print("The testing MAPE for the ARMA model is:",MAPE(test_arima,ARIMA_testing_predictions.Radiation))
print("The testing RMSE for the ARMA model is:",mean_squared_error(test_arima,ARIMA_testing_predictions.Radiation, squared=False))

In [ ]:
#7 day prediction
model = ARIMA(np.transpose(train_arima), order=(1,0,1))
model_fit = model.fit()
ARIMA_7day_predictions=model_fit.forecast(7)
ARIMA_7day_predictions

In [ ]:
plt.plot(ARIMA_7day_predictions, label="Model")
plt.plot(test_arima[0:7], label="Actual")
plt.title('7 Day predictions ARIMA')
plt.xlabel('Date')
plt.xticks(rotation = 45)
plt.ylabel('Radiation')
plt.legend(loc='best')
plt.show()

In [ ]:
print("The 7 day testing MAPE for the ARMA model is:",MAPE(test_arima[0:7],ARIMA_7day_predictions))
print("The 7 day testing RMSE for the ARMA model is:",mean_squared_error(test_arima[0:7],ARIMA_7day_predictions, squared=False))

# Exponential Smoothing

In [ ]:
#exponential smoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
modelexp = SimpleExpSmoothing(train_arima)

In [ ]:
model_fitexp = modelexp.fit()

In [ ]:
EXP_training_predictions=model_fitexp.predict(start=0, end=len(train_arima)-1)

In [ ]:
plt.plot(EXP_training_predictions, label="Model")
plt.plot(train_arima, label="Actual")
plt.title('Training Predictions Exponential Smoothing')
plt.xlabel('Date')
plt.xticks(rotation = 45)
plt.ylabel('Radiation')
plt.legend(loc='best')
plt.show()

In [ ]:
print("The training MAPE for the Exponential Smoothing model is:",MAPE(train_arima,EXP_training_predictions))
print("The training RMSE for the Exponential Smoothing model is:",mean_squared_error(train_arima,EXP_training_predictions, squared=False))

In [ ]:
history = [x for x in np.transpose(train_arima)]
predictions = list()
for t in range(len(np.transpose(test_arima))):
    model = SimpleExpSmoothing(history)
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test_arima[t]
    history.append(obs)#obs originally
    #print('predicted=%f, expected=%f' % (yhat, obs))
EXP_testing_predictions= pd.DataFrame(predictions,columns = ['Radiation'], index= test_arima.index)

In [ ]:
plt.plot(EXP_testing_predictions, label="Model")
plt.plot(test_arima, label="Actual")
plt.title('Testing Predictions Exponential Smoothing')
plt.xlabel('Date')
plt.xticks(rotation = 45)
plt.ylabel('Radiation')
plt.legend(loc='best')
plt.show()

In [ ]:
print("The testing MAPE for the Exponential Smoothing model is:",MAPE(test_arima,EXP_testing_predictions.Radiation))
print("The testing RMSE for the Exponential Smoothing model is:",mean_squared_error(test_arima,EXP_testing_predictions.Radiation, squared=False))

In [ ]:
#7day prediction
model = SimpleExpSmoothing(train_arima)
model_fit = model.fit()
EXP_7day_predictions=model_fit.forecast(len(test_arima))
EXP_7day_predictions

In [ ]:
plt.plot(EXP_7day_predictions[0:7], label="Model")
plt.plot(test_arima[0:7], label="Actual")
plt.title('7 Day Predictions Exponential Smoothing')
plt.xlabel('Date')
plt.xticks(rotation = 45)
plt.ylabel('Radiation')
plt.legend(loc='best')
plt.show()

In [ ]:
print("The 7 day testing MAPE for the EXP model is:",MAPE(test_arima[0:7],EXP_7day_predictions[0:7]))
print("The 7 day testing RMSE for the EXP model is:",mean_squared_error(test_arima[0:7],EXP_7day_predictions[0:7], squared=False))

In [ ]:
plt.plot(EXP_7day_predictions, label="Model")
plt.plot(test_arima, label="Actual")
plt.title('Testing Predictions Exponential Smoothing with no ground truth')
plt.xlabel('Date')
plt.xticks(rotation = 45)
plt.ylabel('Radiation')
plt.legend(loc='best')
plt.show()

In [ ]:
print("The testing MAPE with no ground truth for the EXP model is:",MAPE(test_arima,EXP_7day_predictions))
print("The testing RMSE with no ground truth for the EXP model is:",mean_squared_error(test_arima,EXP_7day_predictions, squared=False))

# LSTM

In [ ]:
##LSTM
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)
 

In [ ]:
# fix random seed for reproducibility
tf.random.set_seed(7)
# load the dataset
dataset = df_final.values
dataset = np.array(df_final.astype('float32'))
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset.reshape(-1, 1))
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
# calculate root mean squared error
# training and testing
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))


In [ ]:
#MAPE training and testing
print('Training Score: MAPE', MAPE(trainY[0], trainPredict[:,0]))
print('Testing Score: MAPE', MAPE(testY[0], testPredict[:,0]))

In [ ]:
#7Days no ground truth testing
pre=testX[0]
youtpred=np.zeros(len(testX))
for i in range(len(testX)):
  outpred=model.predict([pre])
  youtpred[i]=outpred
  pre=outpred
LSTM_out_pred=scaler.inverse_transform(youtpred.reshape(-1, 1))

In [ ]:
# 7 days MAPE and RMSE
print('Testing Score: MAPE', MAPE(testY[0][0:7], LSTM_out_pred[0:7]))
test7Score = np.sqrt(mean_squared_error(testY[0][0:7], LSTM_out_pred[0:7]))
print('Test 7day Score: %.2f RMSE' % (test7Score))

In [ ]:
plt.plot(train_arima[1:-1].index, trainPredict[:,0], label="Model")
plt.plot(train_arima[1:-1].index,trainY[0], label="Actual")
plt.title('Training Predictions LSTM')
plt.xticks(rotation = 45)
plt.xlabel('Date')
plt.ylabel('Radiation')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.plot(test_arima[1:-1].index, testPredict[:,0], label="Model")
plt.plot(test_arima[1:-1].index,testY[0], label="Actual")
plt.title('Testing Predictions LSTM')
plt.xticks(rotation = 45)
plt.xlabel('Date')
plt.ylabel('Radiation')
plt.legend(loc='best')
plt.show()

In [ ]:
day7lstm=LSTM_out_pred[0:7]

plt.plot(test_arima[0:7].index, day7lstm, label="forecast")
plt.plot(test_arima[0:7], label="Actual")
plt.title('7 Day Predictions LSTM')
plt.xticks(rotation = 45)
plt.xlabel('Date')
plt.ylabel('Radiation')
plt.legend(loc='best')
plt.show()

# NeuralProphet

In [ ]:
##pip install neuralprophet
from neuralprophet import NeuralProphet

In [ ]:
nuo=pd.DataFrame(train_arima)
nuo["ds"]= nuo.index
nuo['ds'] = pd.DatetimeIndex(nuo['ds'])
nuo = nuo.rename(columns={'Radiation': 'y'})

nuo_test=pd.DataFrame(test_arima)
nuo_test["ds"]= nuo_test.index
nuo_test['ds'] = pd.DatetimeIndex(nuo_test['ds'])
nuo_test = nuo_test.rename(columns={'Radiation': 'y'})

In [ ]:
m = NeuralProphet(weekly_seasonality=True)
metrics = m.fit(nuo, freq='D', validation_df=nuo_test)


In [ ]:
future = m.make_future_dataframe(nuo_test, periods=0, n_historic_predictions=len(nuo))
forecast = m.predict(future)

In [ ]:
#7 Day testing
plt.plot(test_arima[0:7].index,forecast["yhat1"][0:7], label="Model")
plt.plot(test_arima[0:7], label="Actual")
plt.title('7 Day Predictions Neural Prophet')
plt.xlabel('Date')
plt.xticks(rotation = 45)
plt.ylabel('Radiation')
plt.legend(loc='best')
plt.show()

In [ ]:
print('Testing neural prophet Score: MAPE', MAPE(forecast["y"][0:7], forecast["yhat1"][0:7]))
print("Testing neural prophet Score: RMSE",mean_squared_error(forecast["y"][0:7], forecast["yhat1"][0:7], squared=False))

In [ ]:
# Full testing with no ground truth
print('Testing neural prophet Score: MAPE', MAPE(forecast["y"], forecast["yhat1"]))
print("Testing neural prophet Score: RMSE",mean_squared_error(forecast["y"], forecast["yhat1"], squared=False))

In [ ]:
plt.plot(test_arima.index,forecast["yhat1"], label="Model")
plt.plot(test_arima, label="Actual")
plt.title('Testing Predictions Neural Prophet with no ground truth')
plt.xlabel('Date')
plt.xticks(rotation = 45)
plt.ylabel('Radiation')
plt.legend(loc='best')
plt.show()

In [ ]:
#Predicting the training set
future2 = m.make_future_dataframe(nuo, periods=0, n_historic_predictions=len(nuo))
forecast2 = m.predict(future2)

In [ ]:
#Training Prediction plots
plt.plot(train_arima.index,forecast2["yhat1"], label="Model")
plt.plot(train_arima, label="Actual")
plt.title('Training Predictions Neural Prophet')
plt.xlabel('Date')
plt.xticks(rotation = 45)
plt.ylabel('Radiation')
plt.legend(loc='best')
plt.show()

In [ ]:
print('Training neural prophet Score: MAPE', MAPE(forecast2["y"], forecast2["yhat1"]))
print("Training neural prophet Score: RMSE:",mean_squared_error(forecast2["y"], forecast2["yhat1"], squared=False))